In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)

In [ ]:
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone()
index = pc.Index("faqs")

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.similarity_search("forget password", k=3)

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)
retriever.invoke("how can i stay healthy?")

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "lambda_mult": 0.5},
)
retriever.invoke("how can i stay healthy?")

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "Forget"
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.5}
    ),
    llm=llm,
)

In [ ]:
import torch
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)


from pinecone import Pinecone

pc = Pinecone()
index = pc.Index("faqs")


from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

from langchain.chat_models import init_chat_model
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.5}
    ),
    llm=llm,
)

In [ ]:
query = "forget"

In [ ]:
retriever.invoke(query)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [ ]:
"ff"

: 

In [9]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

E0000 00:00:1759297795.171370  464243 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [21]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

a = llm.invoke(f"i forget password you are bad person\n out should be in {Classification.model_json_schema()} ")

In [22]:
print(a.content)

```json
{
  "sentiment": "sad",
  "aggressiveness": 5,
  "language": "english"
}
```


In [26]:
Classification(**{
  "sentiment": "sad",
  "aggressiveness": 5,
  "language": "english"
})

Classification(sentiment='sad', aggressiveness=5, language='english')

In [69]:
from langchain.chat_models import init_chat_model
# from langchain.output_parsers import EnumOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

from enum import Enum

from langchain.output_parsers import OutputFixingParser
from pydantic import BaseModel, Field, conint


class CategoriesProbabilities(BaseModel):
    Desktop: float = Field(
        ...,
        description="Probability that the ticket relates to general desktop issues, including software, settings, or performance.",
    )
    Desktop_Application: float = Field(
        ...,
        description="Probability that the ticket is about problems with installed desktop applications (e.g., MS Office, browsers).",
    )
    Desktop_Asset_Management: float = Field(
        ...,
        description="Probability that the ticket concerns desktop asset tracking, inventory, or allocation.",
    )
    Desktop_Hardware: float = Field(
        ...,
        description="Probability that the ticket is related to physical hardware issues such as monitor, keyboard, or peripherals.",
    )
    Desktop_Login_Issue: float = Field(
        ...,
        description="Probability that the ticket is about desktop login problems, like incorrect credentials or account lockouts.",
    )
    Desktop_Storage: float = Field(
        ...,
        description="Probability that the ticket involves local storage issues, such as low disk space or drive access problems.",
    )

    Email: float = Field(
        ...,
        description="Probability that the ticket relates to general email issues, including access, sync, or configuration problems.",
    )
    Email_Email_Create: float = Field(
        ...,
        description="Probability that the ticket is a request to create a new company email ID or mailbox.",
    )
    Email_Whitelist_Email: float = Field(
        ...,
        description="Probability that the ticket requests to whitelist an email address or domain to prevent blocking or spam issues.",
    )

    Internet: float = Field(
        ...,
        description="Probability that the ticket relates to internet connectivity issues, such as no access, slow speed, or frequent disconnections.",
    )
    Network: float = Field(
        ...,
        description="Probability that the ticket involves internal network problems, including LAN/Wi-Fi issues or access to shared resources.",
    )
    International_Calling: float = Field(
        ...,
        description="Probability that the ticket relates to enabling, disabling, or troubleshooting international calling services.",
    )

    Server: float = Field(
        ...,
        description="Probability that the ticket is about general server-related issues or requests not covered by subcategories.",
    )
    Server_Application: float = Field(
        ...,
        description="Probability that the ticket concerns applications hosted on servers, including downtime or access errors.",
    )
    Server_Client_Servers: float = Field(
        ...,
        description="Probability that the ticket relates to client-specific servers, configurations, or deployment problems.",
    )

    Storage: float = Field(
        ...,
        description="Probability that the ticket involves shared or cloud storage issues (e.g., OneDrive, NAS, shared drives).",
    )
    Printing: float = Field(
        ...,
        description="Probability that the ticket is about printing issues, such as printer detection, queue errors, or print quality problems.",
    )

    Website_Blocked: float = Field(
        ...,
        description="Probability that the ticket requests access to a blocked website or restricted web content.",
    )
    New_Service_Request: float = Field(
        ...,
        description="Probability that the ticket is a request for a new IT service, software installation, or hardware provisioning.",
    )

    CSG_PDG: float = Field(
        ...,
        description="Probability that the ticket is related to Corporate Services Group – Product Development Group internal tools or systems.",
    )
    Others: float = Field(
        ...,
        description="Probability that the ticket does not fall into any predefined categories and needs manual review or reassignment.",
    )

    def top_category(self) -> tuple[str, float]:
        data = self.model_dump()  # Pydantic v2
        return max(data.items(), key=lambda kv: kv[1])


class Priority(Enum):
    VeryHigh = 1
    High = 2
    Normal = 3
    Low = 4
    VeryLow = 5


class TicketPrediction(BaseModel):
    categories_probabilities: CategoriesProbabilities = Field(
        ..., description="Probability scores for each ticket category."
    )
    priority: Priority = Field(
        ...,
        description=(
            "Priority of the ticket, from 1 to 5:\n"
            "1 = Very High\n"
            "2 = High\n"
            "3 = Normal\n"
            "4 = Low\n"
            "5 = Very Low"
        ),
    )


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=TicketPrediction)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

prompt = PromptTemplate(
    template="""
You are an IT support ticket classifier.

Given the ticket description, provide the following:

1. **categories_probabilities**: a probability score (in percentage) for each ticket category that represents how likely the ticket belongs to that category.
   - All probabilities must sum up to 100%.
   - No two categories should have the same probability score.

2. **priority**: one of the following Enum values (VeryHigh = 1, High = 2, Normal = 3, Low = 4, VeryLow = 5), representing how important it is to resolve the issue quickly so that there are no work blockers or wasted time.

{format_instructions}

Ticket Description:
"{query}"
""",
    input_variables=["query"],
    # partial_variables={"format_instructions": new_parser.get_format_instructions()},
    partial_variables={"format_instructions": new_parser.get_format_instructions()},
)


# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "my active directory is locked"})
opt = parser.invoke(output)
print(opt.categories_probabilities.top_category())
print(opt.priority)

E0000 00:00:1759737521.519817 2987278 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


('Desktop_Login_Issue', 93.94)
Priority.VeryHigh


In [70]:
opt.categories_probabilities.top_category()

('Desktop_Login_Issue', 93.94)

In [42]:
print(prompt.invoke({"query": "it is very good  my  bloody son."}).to_string())


You are an IT support ticket classifier.

Given the ticket description, provide the following:

1. **categories_probabilities**: a probability score (in percentage) for each ticket category that represents how likely the ticket belongs to that category.
   - All probabilities must sum up to 100%.
   - No two categories should have the same probability score.

2. **priority**: one of the following Enum values (VeryHigh = 1, High = 2, Normal = 3, Low = 4, VeryLow = 5), representing how important it is to resolve the issue quickly so that there are no work blockers or wasted time.

Output the result in JSON format compatible with the `TicketPrediction` Pydantic model.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted ins

In [2]:
from support_hub.ticket_classifier.ticket_classification import run_ticket_classification

await run_ticket_classification("create new s3 bucket for me?")

E0000 00:00:1759818928.521625 3489210 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759818928.543623 3489210 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'ticket_class': {'service': {'service_name': '', 'service_id': '2'},
  'role': 'cloud engineer',
  'type': {'type_name': '', 'type_id': '5'}},
 'priority': 3}

In [1]:
from support_hub.ticket_classifier.ticket_classification import run_ticket_classification

await run_ticket_classification("my active directory is locked")

E0000 00:00:1759818907.220970 3489210 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759818907.267192 3489210 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'ticket_class': {'service': {'service_name': 'Desktop Login',
   'service_id': '18'},
  'role': 'Manager',
  'type': {'type_name': '', 'type_id': '5'}},
 'priority': 1}

In [9]:
import asyncio
import aiohttp
import json

async def fetch_skills(session, base_url):
    """
    Asynchronously fetch skills data from the API endpoint.
    
    Args:
        session (aiohttp.ClientSession): The aiohttp session for making requests.
        base_url (str): The base URL of the API (e.g., 'http://example.com').
    
    Returns:
        dict: The parsed JSON response.
    """
    url = f"{base_url}/api/skills/"
    async with session.get(url) as response:
        # if response.status == 200:
            return await response.json()
        # else:
        #     raise Exception(f"Failed to fetch data: HTTP {response.status}")

async with aiohttp.ClientSession() as session:
       
            data = await fetch_skills(session, "http://192.168.71.115")
            print(data)

{'status': True, 'status_code': 200, 'message': 'List fetched successfully', 'data': [{'role': 'Computer Hardware Engineer'}, {'role': 'Linux Admin'}, {'role': 'Manager'}, {'role': 'cloud engineer'}]}


In [ ]:
import aiohttp

async def fetch_roles(session, base_url):
    """
    Asynchronously fetch skills data from the API endpoint.
    
    Args:
        session (aiohttp.ClientSession): The aiohttp session for making requests.
        base_url (str): The base URL of the API (e.g., 'http://example.com').
    
    Returns:
        dict: The parsed JSON response.
    """
    url = f"{base_url}/api/skills/"
    async with session.get(url) as response:
        if response.status == 200:
            return await response.json()
        else:
            raise Exception(f"Failed to fetch data: HTTP {response.status}")

async def list_roles():
    base_url = "http://192.168.71.115"  # Replace with your actual base URL
    async with aiohttp.ClientSession() as session:
        try:
            data = await fetch_roles(session, base_url)
            # Example: Access the roles
            roles = [item["role"] for item in data["data"]]
            return {"roles": roles}
        except Exception as e:
            print(f"Error: {e}")
await list_roles()

{'roles': ['Computer Hardware Engineer',
  'Linux Admin',
  'Manager',
  'cloud engineer']}

In [24]:
roles = await list_roles()

In [25]:
roles

{'roles': ['Computer Hardware Engineer',
  'Linux Admin',
  'Manager',
  'cloud engineer']}

In [27]:
import aiohttp


class RoleFetcher:
    """
    A class to asynchronously fetch roles from the API.
    """

    def __init__(self, base_url: str = "http://192.168.71.115"):
        """
        Initialize the RoleFetcher with the base URL of the API.

        Args:
            base_url (str): The base URL of the API (e.g., 'http://192.168.71.115').
        """
        self.base_url = base_url

    async def fetch_roles(self):
        """
        Asynchronously fetch roles from the API endpoint.

        Returns:
            dict: A dictionary containing the list of roles.

        Raises:
            Exception: If the API request fails.
        """
        url = f"{self.base_url}/api/skills/"
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    roles = [item["role"] for item in data["data"]]
                    return {"roles": roles}
                else:
                    raise Exception(f"Failed to fetch data: HTTP {response.status}")

    async def list_roles(self):
        """
        Asynchronously list roles from the API.

        Returns:
            dict: A dictionary containing the list of roles.

        Raises:
            Exception: If the API request fails.
        """
        return await self.fetch_roles()


fetcher = RoleFetcher()

await fetcher.list_roles()

{'roles': ['Computer Hardware Engineer',
  'Linux Admin',
  'Manager',
  'cloud engineer']}

In [30]:
from pydantic import BaseModel, Field

from support_hub.ticket_classifier.models.utils import RoleFetcher


async def get_roles():
    fetcher = RoleFetcher()
    try:
        result = await fetcher.list_roles()
        return result.get("roles", [])
    except Exception as e:
        print(f"Error: {e}")
a = await get_roles()

class Roles(BaseModel):
    role_names: list[str] = Field(default=a, description="Names of the roles")


In [31]:
Roles(role_names=a)

Roles(role_names=['Computer Hardware Engineer', 'Linux Admin', 'Manager', 'cloud engineer'])

In [ ]:
from typing import Literal



In [35]:
a

typing.Literal[['i', 'am', 'sourabh']]

In [37]:
from support_hub.ticket_classifier.parsers.output_parsers import get_pydantic_role_parser

p = get_pydantic_role_parser()

In [43]:
from support_hub.ticket_classifier.prompts.ticket_prompt import get_ticket_classification_prompt_roles

print(get_ticket_classification_prompt_roles(p.get_format_instructions()).invoke({"ticket": "my active directory is locked"}).to_string())


You are an IT support ticket classifier.

Given the ticket, provide the following:

role: a single resolver role to assign this ticket to.
   - Return exactly one string that matches character-for-character one of the provided roles.
   - Do not invent or modify role names.
   - If multiple roles could apply, choose the most specialized technical role aligned with the primary action implied by the ticket.
   - If unable to confidently select an exact match from the provided roles, return "Manager" by default.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Roles": {"enum": ["C